In [10]:
#Importación de librearías necesarias
import pandas as pd
from datetime import datetime
import socket
import numpy as np
import math
import time
import pickle  #Para guardar archivos
import os

from pympler import asizeof #Para liberar memoria
import gc

from matplotlib import pyplot as plt
import seaborn as sns

from sklearn.metrics import confusion_matrix
from sklearn.utils.multiclass import unique_labels

from sklearn.model_selection import train_test_split

from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from scipy.stats import randint as sp_randint

from sklearn.svm import SVC 
from xgboost import XGBClassifier

from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.utils import to_categorical

import joblib

semilla = 8

In [2]:
#Path general de archivos
if socket.gethostname()=='LAPTOP-PUSGG08B': #Ip de la laptop
    ruta = "E:/Cristian Guatemal/Master/Big Data y Ciencia de Datos/VIU_TFM/Data/TFM/"
    r_ruta = "E:/Cristian Guatemal/Master/Big Data y Ciencia de Datos/VIU_TFM/RData/TFM/"
elif socket.gethostname()=='PCUIOMTDAIE6382': #Ip del working pc
    ruta =   "D:/Master/Big_Data_Ciencia_Datos/VIU_TFM/Data/TFM/"
    r_ruta = "D:/Master/Big_Data_Ciencia_Datos/VIU_TFM/RData/TFM/"
elif socket.gethostname()=='LPUIODAIE208Y3X': #Ip del working laptop
    ruta =   "C:/Cristian_Guatemal/Personal/Master/Big_Data_Ciencia_Datos/VIU_TFM/Data/TFM/"
    r_ruta = "C:/Cristian_Guatemal/Personal/Master/Big_Data_Ciencia_Datos/VIU_TFM/RData/TFM/"
# Ruta del archivo de pensionistas de vejez
ruta_vj = ruta + 'POB_VEJ_CD656_NEW.dsv'
# Ruta del archivo de historia laboral de pensionistas
ruta_afi = ruta + 'APORTES_CD656_new.dsv'

In [3]:
# Cargar archivo------------------------------------------------------------------------------------------------------------
directorio = r_ruta
nombre_archivo = 'viu_ml_preparacion_data.pkl'
ruta_archivo = os.path.join(directorio, nombre_archivo)
data= pd.read_pickle(ruta_archivo)

In [4]:
data

,SEXO,NUMERO_IMPOSICIONES,BASE_CAL,N_MESES,PRES,LS2,SAL_PROM2,LS_MS,TIEM_T,TIEM_MA,...,N_IND_P,N_VOL_EC_P,N_VOL_EX_P,M_PRI_P,M_PUB_P,M_IND_P,M_VOL_EC_P,M_VOL_EX_P,EDAD_J,TARGET
0,1,288.0,210.635000,297.0,2,382.4950,201.720411,370.00000,85,60,...,0.0,0.000000,0.0,1.000000,0.000000,0.0,0.000000,0.0,72,0
1,1,514.0,508.824667,523.0,2,1074.1950,499.528730,985.98375,79,60,...,0.0,0.000000,0.0,0.000000,1.000000,0.0,0.000000,0.0,64,0
2,0,544.0,566.339833,547.0,2,793.7350,357.121633,884.29750,83,60,...,0.0,0.000000,0.0,0.361702,0.638298,0.0,0.000000,0.0,68,0
3,1,278.0,1285.000000,278.0,2,1950.0000,851.960317,2950.00000,74,60,...,0.0,0.000000,0.0,1.000000,0.000000,0.0,0.000000,0.0,65,1
4,0,481.0,1325.717667,481.0,2,3056.1000,1065.141687,3341.93000,75,60,...,0.0,0.000000,0.0,0.000000,1.000000,0.0,0.000000,0.0,62,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
442974,1,120.0,399.333333,121.0,2,421.0000,397.121212,409.00000,93,60,...,0.0,0.848485,0.0,0.166667,0.000000,0.0,0.833333,0.0,91,0
442975,0,109.0,424.344000,110.0,1,455.0375,382.017182,423.53125,68,60,...,0.0,0.000000,0.0,1.000000,0.000000,0.0,0.000000,0.0,67,1
442976,0,68.0,579.554912,69.0,1,904.4500,564.833623,909.81000,78,60,...,0.0,0.000000,0.0,1.000000,0.000000,0.0,0.000000,0.0,70,0
442977,1,180.0,215.797833,87.0,2,335.0000,196.642184,300.00000,78,60,...,0.0,0.000000,0.0,1.000000,0.000000,0.0,0.000000,0.0,66,0


In [4]:
#Selección de variables para entrenar los modelos
val_otr = ['BASE_CAL', 'LS2', 'SAL_PROM2', 'LS_MS', 'SUELDO',
           'SEXO', 'PRES', 'N_PRI_P', 'N_PUB_P', 'N_IND_P', 'N_VOL_EC_P',
           'N_VOL_EX_P', 'M_PRI_P', 'M_PUB_P', 'M_IND_P', 'M_VOL_EC_P','M_VOL_EX_P',
           'NUMERO_IMPOSICIONES', 'TIEM_T',
           'TARGET']
data_otr = data[ val_otr ]
data_otr

,BASE_CAL,LS2,SAL_PROM2,LS_MS,SUELDO,SEXO,PRES,N_PRI_P,N_PUB_P,N_IND_P,N_VOL_EC_P,N_VOL_EX_P,M_PRI_P,M_PUB_P,M_IND_P,M_VOL_EC_P,M_VOL_EX_P,NUMERO_IMPOSICIONES,TIEM_T,TARGET
0,210.635000,382.4950,201.720411,370.00000,14725.59,1,2,1.000000,0.000000,0.0,0.000000,0.0,1.000000,0.000000,0.0,0.000000,0.0,288.0,85,0
1,508.824667,1074.1950,499.528730,985.98375,31470.31,1,2,0.000000,1.000000,0.0,0.000000,0.0,0.000000,1.000000,0.0,0.000000,0.0,514.0,79,0
2,566.339833,793.7350,357.121633,884.29750,34997.92,0,2,0.357143,0.642857,0.0,0.000000,0.0,0.361702,0.638298,0.0,0.000000,0.0,544.0,83,0
3,1285.000000,1950.0000,851.960317,2950.00000,107347.00,1,2,1.000000,0.000000,0.0,0.000000,0.0,1.000000,0.000000,0.0,0.000000,0.0,278.0,74,1
4,1325.717667,3056.1000,1065.141687,3341.93000,88406.76,0,2,0.000000,1.000000,0.0,0.000000,0.0,0.000000,1.000000,0.0,0.000000,0.0,481.0,75,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
442974,399.333333,421.0000,397.121212,409.00000,26210.00,1,2,0.151515,0.000000,0.0,0.848485,0.0,0.166667,0.000000,0.0,0.833333,0.0,120.0,93,0
442975,424.344000,455.0375,382.017182,423.53125,42021.89,0,1,1.000000,0.000000,0.0,0.000000,0.0,1.000000,0.000000,0.0,0.000000,0.0,109.0,68,1
442976,579.554912,904.4500,564.833623,909.81000,38973.52,0,1,1.000000,0.000000,0.0,0.000000,0.0,1.000000,0.000000,0.0,0.000000,0.0,68.0,78,0
442977,215.797833,335.0000,196.642184,300.00000,17107.87,1,2,1.000000,0.000000,0.0,0.000000,0.0,1.000000,0.000000,0.0,0.000000,0.0,180.0,78,0


In [5]:
train, test = train_test_split( data_otr, test_size = 0.2, random_state = semilla )
train.reset_index(drop=True, inplace= True)
test.reset_index(drop=True, inplace= True)

In [7]:
train

,BASE_CAL,LS2,SAL_PROM2,LS_MS,SUELDO,SEXO,PRES,N_PRI_P,N_PUB_P,N_IND_P,N_VOL_EC_P,N_VOL_EX_P,M_PRI_P,M_PUB_P,M_IND_P,M_VOL_EC_P,M_VOL_EX_P,NUMERO_IMPOSICIONES,TIEM_T,TARGET
0,450.145333,578.940,385.783908,450.00000,67126.40,1,2,1.000000,0.000000,0.000000,0.000000,0.0,1.00000,0.000000,0.000000,0.0,0.0,360.0,61,1
1,1545.193333,3221.700,1058.564771,1805.00000,115383.56,1,2,0.000000,1.000000,0.000000,0.000000,0.0,0.00000,1.000000,0.000000,0.0,0.0,414.0,71,0
2,2071.098000,2766.090,1447.054810,3159.72875,303881.51,0,2,0.871429,0.000000,0.000000,0.128571,0.0,1.00000,0.000000,0.000000,0.0,0.0,433.0,61,0
3,525.298833,875.000,386.323246,554.50000,73787.74,0,2,0.000000,1.000000,0.000000,0.000000,0.0,0.00000,1.000000,0.000000,0.0,0.0,407.0,71,0
4,403.133333,545.245,337.471017,409.00000,59732.37,1,2,1.000000,0.000000,0.000000,0.000000,0.0,1.00000,0.000000,0.000000,0.0,0.0,186.0,66,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
354211,445.387333,650.000,284.338207,975.00000,41229.04,0,2,0.200000,0.524138,0.275862,0.000000,0.0,0.47541,0.393443,0.131148,0.0,0.0,422.0,68,1
354212,148.312667,242.135,145.520635,242.13500,9167.80,1,2,0.000000,0.000000,1.000000,0.000000,0.0,0.00000,0.000000,1.000000,0.0,0.0,262.0,81,0
354213,550.860833,883.470,429.636082,600.60000,73467.77,1,2,1.000000,0.000000,0.000000,0.000000,0.0,1.00000,0.000000,0.000000,0.0,0.0,342.0,72,0
354214,375.611667,552.500,312.598658,427.65000,46577.20,0,0,1.000000,0.000000,0.000000,0.000000,0.0,1.00000,0.000000,0.000000,0.0,0.0,307.0,84,1


In [8]:
test

,BASE_CAL,LS2,SAL_PROM2,LS_MS,SUELDO,SEXO,PRES,N_PRI_P,N_PUB_P,N_IND_P,N_VOL_EC_P,N_VOL_EX_P,M_PRI_P,M_PUB_P,M_IND_P,M_VOL_EC_P,M_VOL_EX_P,NUMERO_IMPOSICIONES,TIEM_T,TARGET
0,395.550500,414.000,268.144690,414.00000,30300.35,0,2,1.000000,0.000000,0.0,0.000000,0.0,1.000000,0.000000,0.0,0.0,0.0,189.0,76,1
1,707.392000,1108.330,566.876842,700.00000,118477.26,0,2,1.000000,0.000000,0.0,0.000000,0.0,1.000000,0.000000,0.0,0.0,0.0,443.0,63,1
2,294.541500,634.015,280.728696,598.94875,19370.28,0,2,0.000000,1.000000,0.0,0.000000,0.0,0.000000,1.000000,0.0,0.0,0.0,300.0,81,0
3,1412.000000,2861.255,987.774130,1412.00000,181750.44,1,2,0.000000,1.000000,0.0,0.000000,0.0,0.000000,1.000000,0.0,0.0,0.0,457.0,66,0
4,2472.000000,5000.700,1666.420637,2472.00000,339949.81,0,2,0.000000,0.990196,0.0,0.009804,0.0,0.000000,1.000000,0.0,0.0,0.0,336.0,70,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88549,370.974167,488.015,328.602000,445.86500,31217.19,0,2,0.915789,0.084211,0.0,0.000000,0.0,0.875000,0.125000,0.0,0.0,0.0,126.0,77,1
88550,1339.639000,2855.000,866.709699,2317.50000,115272.39,0,2,0.037594,0.962406,0.0,0.000000,0.0,0.083333,0.916667,0.0,0.0,0.0,425.0,67,0
88551,825.662500,826.000,756.968112,797.50000,148365.75,1,2,1.000000,0.000000,0.0,0.000000,0.0,1.000000,0.000000,0.0,0.0,0.0,578.0,75,1
88552,438.682667,675.500,334.648627,615.74000,34134.16,0,2,1.000000,0.000000,0.0,0.000000,0.0,1.000000,0.000000,0.0,0.0,0.0,144.0,79,1


In [6]:
def plot_confusion_matrix(y_true, y_pred, classes,
                          normalize=False,
                          title=None,
                          cmap=plt.cm.Blues):
    """
    This function prints and pljots the confusion matrix.
    Normalization can be applied by setting normalize=True.
    """
    if not title:
        if normalize:
            title = 'Normalized confusion matrix'
        else:
            title = 'Confusion matrix, without normalization'

    # Compute confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    # Only use the labels that appear in the data
    classes = classes[unique_labels(y_true, y_pred)]
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    fig, ax = plt.subplots()
    im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
    ax.figure.colorbar(im, ax=ax)
    # We want to show all ticks...
    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           # ... and label them with the respective list entries
           xticklabels=classes, yticklabels=classes,
           title=title,
           ylabel='True label',
           xlabel='Predicted label')

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
             rotation_mode="anchor")

    # Loop over data dimensions and create text annotations.
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], fmt),
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black")
    fig.tight_layout()
    return ax

## SVC

In [10]:
model_svc = SVC( C=1E10, random_state = semilla)

In [11]:
param_dist_r_svc = { "C":[1,5,10,15],
                     "kernel":['poly', 'rbf', 'sigmoid', 'precomputed'],
                     "gamma":['scale', 'auto'], 
                     "class_weight":['balanced']
                    
                   }

In [ ]:
inicio = time.time()
random_svc = RandomizedSearchCV( estimator = model_svc, 
                                 param_distributions = param_dist_r_svc,
                                 n_iter = 100, cv = 5, scoring = 'accuracy', random_state = semilla, n_jobs = -1)

# Fit the random search model
random_svc.fit(X = train.drop(['TARGET'], axis=1), y = train['TARGET'])

fin = time.time()  
tm= fin-inicio
print('Tiempo de ejecución es:',tm//3600,'horas con',tm%3600//60,'minutos y', tm%60,'segundos' )

C:\ProgramData\anaconda3\Lib\site-packages\sklearn\model_selection\_search.py:307: UserWarning: The total space of parameters 32 is smaller than n_iter=100. Running 32 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


In [ ]:
random_svc.best_estimator_

In [ ]:
best_random_svc = random_svc.best_estimator_
best_random_svc.fit( X = train.drop(['TARGET'], axis=1), y = train['TARGET'])
y_pred_r_svc= best_random_svc.predict(X = test.drop(['TARGET'], axis = 1,))
acc = accuracy_score(test['TARGET'], y_pred_r_svc )
print ('Acc', acc)

In [ ]:
print(classification_report(test['TARGET'], y_pred_r_svc) )
plot_confusion_matrix(test['TARGET'], y_pred_r_svc, normalize=True, classes=best_random_sv.classes_, title='Confusion matrix')

In [ ]:
# FEATURE RELEVANCIES
pd.DataFrame({'Attributes': train.drop('TARGET', axis=1).columns.tolist(),
              'Importancia': best_random_svc.feature_importances_}).sort_values( by='Importancia', ascending=False).reset_index(drop=True)

In [ ]:
#Gua9rdando el modelo
joblib.dump( best_random_svc, r_ruta + 'SVC_random_search.joblib')

In [ ]:
#GridSearch
param_dist_g_svc = { "C":[1,5,10,15],
                     "kernel":['linear'],
                     "gamma":['scale'], 
                     "class_weight":['balanced']
                   }

In [ ]:
inicio = time.time()
grid_svc = GridSearchCV( estimator = model_svc,
                         param_grid = param_dist_g_svc, 
                         cv=5,  scoring = 'accuracy', n_jobs = -1 )
grid_svc.fit( X = train.drop(['TARGET'], axis=1), y = train['TARGET'] )
fin = time.time()  
tm= fin-inicio
print('Tiempo de ejecución es:',tm//3600,'horas con',tm%3600//60,'minutos y', tm%60,'segundos' )

In [ ]:
grid_svc.best_params_

In [ ]:
best_grid_svc = grid_svc.best_estimator_
# fit and predict
best_grid_svc.fit( X = train.drop(['TARGET'], axis=1), y = train['TARGET'])
y_pred_g_svc = best_grid_svc.predict(X = test.drop(['TARGET'], axis = 1,))
acc = accuracy_score(test['TARGET'], y_pred_g_svc )
print ('Acc', acc)

In [ ]:
print(classification_report(test['TARGET'], y_pred_g_svc ) )
plot_confusion_matrix(test['TARGET'], y_pred_g_svc, normalize=True, classes=best_grid_svc.classes_, title='Confusion matrix')

In [ ]:
# FEATURE RELEVANCIES
pd.DataFrame({'Attributes': train.drop('TARGET', axis=1).columns.tolist(),
              'Importancia': best_grid_svc.feature_importances_}).sort_values( by='Importancia', ascending=False).reset_index(drop=True)

In [ ]:
#Guardando el modelo
joblib.dump( best_grid_svc, r_ruta + 'SVC_grid_search.joblib')

## XGBClassifier

In [ ]:
model_xgb = XGBClassifier( random_state = semilla, use_label_encoder=False, eval_metric='mlogloss' )

In [ ]:
param_dis_r_xgb = { 'n_estimators': np.arange(50, 500, 50),  
                    'learning_rate': [0.01, 0.1, 0.2, 0.3],  
                    'max_depth': np.arange(3, 15, 1),  
                    'min_child_weight': np.arange(1, 10, 1),  
                    'subsample': [0.6, 0.8, 1.0],  
                    'colsample_bytree': [0.6, 0.8, 1.0],  
                    'gamma': [0, 0.1, 0.2, 0.3],  
                    }

In [ ]:
inicio = time.time()
random_xgb = RandomizedSearchCV( estimator = model_xgb, 
                                 param_distributions = param_dis_r_xgb,
                                 n_iter = 100, cv = 5, scoring = 'accuracy', random_state = semilla, n_jobs = -1)

# Fit the random search model
random_xgb.fit(X = train.drop(['TARGET'], axis=1), y = train['TARGET'])

fin = time.time()  
tm= fin-inicio
print('Tiempo de ejecución es:',tm//3600,'horas con',tm%3600//60,'minutos y', tm%60,'segundos' )

In [ ]:
random_xgb.best_estimator_

In [ ]:
best_random_xgb = random_xgb.best_estimator_
best_random_xgb.fit( X = train.drop(['TARGET'], axis=1), y = train['TARGET'])
y_pred_r_xgb = best_random_xgb.predict(X = test.drop(['TARGET'], axis = 1,))
acc = accuracy_score(test['TARGET'], y_pred_r_xgb )
print ('Acc', acc)

In [ ]:
print(classification_report(test['TARGET'], y_pred_r_xgb) )
plot_confusion_matrix(test['TARGET'], y_pred_r_xgb, normalize=True, classes=best_random_xgb.classes_, title='Confusion matrix')

In [ ]:
# FEATURE RELEVANCIES
pd.DataFrame({'Attributes': train.drop('TARGET', axis=1).columns.tolist(),
              'Importancia': best_random_xgb.feature_importances_}).sort_values( by='Importancia', ascending=False).reset_index(drop=True)

In [ ]:
#Gua9rdando el modelo
joblib.dump( best_random_xgb, r_ruta + 'XGBClassifier_random_search.joblib')

## Redes Neuronales

In [12]:
train.shape

(354216, 20)

In [14]:
model = Sequential()

# Definimos una capa convolucional
model.add(Conv2D(128, kernel_size=(3, 3), activation='relu', input_shape=(354216,20)))

# Definimos una segunda capa convolucional
model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))

# Definimos una tercera capa convolucional
model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))

# Añadimos nuestro clasificador
model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dense(10, activation='softmax'))

# Compilamos el modelo (actualizamos la sintaxis de `Adam`)
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(learning_rate=0.0001, decay=1e-6),  # Cambiamos `lr` por `learning_rate`
              metrics=['accuracy'])

# Entrenamos el modelo
model.fit(train.drop('TARGET', axis=1), train['TARGET'],
          batch_size=128,
          shuffle=True,
          epochs=10,
          validation_data=(test.drop('TARGET', axis=1), test['TARGET']))

# Evaluamos el modelo
scores = model.evaluate(test.drop('TARGET', axis=1), test['TARGET'])

# Mostramos el resultado de la evaluación
print('Loss: %.3f' % scores[0])
print('Accuracy: %.3f' % scores[1])

C:\Users\cristian.guatemal\AppData\Roaming\Python\Python311\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


ValueError: Input 0 of layer "conv2d_7" is incompatible with the layer: expected min_ndim=4, found ndim=3. Full shape received: (None, 354216, 20)

In [ ]:
#Normaliza
val_no_acp = ['SEXO','PRES',
             'N_PRI_P','N_PUB_P', 'N_IND_P', 'N_VOL_EC_P', 'N_VOL_EX_P', 
             'M_PRI_P','M_PUB_P', 'M_IND_P', 'M_VOL_EC_P', 'M_VOL_EX_P',
             'TARGET']
col_na = train.drop( val_no_acp, axis=1).columns.tolist()

escal_nor = StandardScaler()

Xs = escal_nor.fit_transform( train.drop( val_no_acp, axis=1)) #Se normaliza el train
Xs = pd.concat([ pd.DataFrame(Xs).reset_index().drop(['index'], axis=1), 
                 pd.DataFrame(train[val_no_acp]).reset_index().drop(['index'], axis=1)], axis=1)
Xs.columns = col_na + val_no_acp

test_norm = escal_nor.transform( test.drop( val_no_acp, axis=1))
Ys = pd.concat([ pd.DataFrame(test_norm).reset_index().drop(['index'], axis=1), 
                   pd.DataFrame(test[val_no_acp]).reset_index().drop(['index'], axis=1)], axis=1)
Ys.columns = Xs.columnsndo los datos


In [ ]:
# Inizializamos el modelo
model = Sequential()

# Definimos una capa convolucional
model.add(Conv2D(128, kernel_size=(3, 3), activation='relu', input_shape=(32, 32, 3)))

# Definimos una segunda capa convolucional
model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))

# Definimos una tercera capa convolucional
model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))

# Añadimos nuestro clasificador
model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dense(10, activation='softmax'))

# Compilamos el modelo
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(lr=0.0001, decay=1e-6),
              metrics=['accuracy'])

# Entrenamos el modelo
model.fit(Xs.drop('TARGET', axis = 1), Xs['TARGET]',
          batch_size=128,
          shuffle=True,
          epochs=10,
          validation_data=(Ys.drop('TARGET', axis=1), Ys['TARGET'])) # aquí deberíamos usar un conjunto distinto al de test!!!

# Evaluamos el modelo
scores = model.evaluate( Ys.drop('TARGET', axis=1), Ys['TARGET'] )

print('Loss: %.3f' % scores[0])
print('Accuracy: %.3f' % scores[1])